# Validation of Simple Models
Simple cobra models with 1-2 bacteria, and 1-2 metabolites

Communities will be:
- I- Single Model, Eat A, Lose Bio 
- II- Single Model, Consume A, Lose B, Eat B, Lose Bio
- III- 2 Model, Eat A, Each Lose Bio
- IV- 2 Model, 1 eat A, 2 eat B 
- V- 2 Model, Eat A, 1 lose Bio&B, 2 eat B, Lose Bio

In [1]:
import iifba
import cobra as cb
import numpy as np
import matplotlib.pyplot as plt

## Create Models

In [ ]:
# Sit I, (III-org1, IV-org1)

# Create model
model_I = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product
bio1e = cb.Metabolite("bio1e", compartment="e")  # intracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=-1000, upper_bound=1000)
exBio1.add_metabolites({
    bio1e: -1
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport A_e → A_c
transport_bio1 = cb.Reaction("T_Bio1", lower_bound=0, upper_bound=1000)
transport_bio1.add_metabolites({
    bio1e: 1,
    bio1: -1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass1 = cb.Reaction("BiomassOrg1", lower_bound=0, upper_bound=1000)
biomass1.add_metabolites({
    A_c: -1,
    bio1: 1
})

# Add reactions to the model
model_I.add_reactions([exA, transport, transport_bio1, biomass1,  exBio1])

# Set objective
model_I.objective = "T_Bio1"

# Solve
sol = model_I.optimize()
print(f"Objective direction: {model_I.objective.direction}")
print(f"Max biomass: {sol.objective_value}\n")
print(sol.fluxes)

model_I.summary()

cb.io.save_json_model(model_I, "sim_I.json")
cb.io.save_json_model(model_I, "sim_III_1.json")
cb.io.save_json_model(model_I, "sim_IV_1.json")


Objective direction: max
Max biomass: 10.0

Ex_A           -10.0
T_A             10.0
T_Bio1          10.0
BiomassOrg1     10.0
exchangeBio1    10.0
Name: fluxes, dtype: float64


[<Reaction Ex_A at 0x733b6d042530>, <Reaction exchangeBio1 at 0x733b6d042380>]

In [11]:
# Sit II
# Create model
model_II = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product
bio1e = cb.Metabolite("bio1e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=0, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=0, upper_bound=1000)
exBio1.add_metabolites({
    bio1e: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})
# Transport B_c → B_e
transport_bio1 = cb.Reaction("T_Bio1", lower_bound=0, upper_bound=1000)
transport_bio1.add_metabolites({
    bio1: -1,
    bio1e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes 2 A_c
    bio1: 1,  # Produces bio1
    B_c: 1   # Always produces 1 B_c as byproduct
})
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassB = cb.Reaction("BiomassFromB", lower_bound=0, upper_bound=1000)
biomassB.add_metabolites({
    bio1: 1,  # Produces bio1
    B_c: -1   # Always produces 1 B_c as byproduct
})



# Add reactions to the model (FIXED: Added missing biomass1B!)
model_II.add_reactions([exA, exB, transportA, transportB, transport_bio1, biomassA, biomassB,  exBio1])



# Set objective
model_II.objective = "T_Bio1"

# Solve
sol = model_II.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_II.summary()

cb.io.save_json_model(model_II, "sim_II.json")


Max biomass: 20.0
Ex_A           -10.0
Ex_B             0.0
T_A             10.0
T_B              0.0
T_Bio1          20.0
BiomassFromA    10.0
BiomassFromB    10.0
exchangeBio1    20.0
Name: fluxes, dtype: float64


In [16]:
# sit III org 2

# Create model
model_III = cb.Model("Org2")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product
bio2e = cb.Metabolite("bio2e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Biomass exchange: export bio1
exBio2 = cb.Reaction("exchangeBio2", lower_bound=-1000, upper_bound=1000)
exBio2.add_metabolites({
    bio2e: -1.0
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport A_e → A_c
transport_bio2 = cb.Reaction("T_Bio2", lower_bound=0, upper_bound=1000)
transport_bio2.add_metabolites({
    bio2e: 1,
    bio2: -1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass2 = cb.Reaction("BiomassOrg2", lower_bound=-1000, upper_bound=1000)
biomass2.add_metabolites({
    A_c: -1,
    bio2: 1
})

# Add reactions to the model
model_III.add_reactions([exA, transport, transport_bio2, biomass2,  exBio2])

# Set objective
model_III.objective = "T_Bio2"

# Solve
sol = model_III.optimize()
print(f"Objective direction: {model_III.objective.direction}")
print(sol.fluxes)
model_III.summary()

cb.io.save_json_model(model_III, "sim_III_2.json")


Objective direction: max
Ex_A           -10.0
T_A             10.0
T_Bio2          10.0
BiomassOrg2     10.0
exchangeBio2    10.0
Name: fluxes, dtype: float64


In [17]:
# sit. IV org 2

# Create model
model_IV = cb.Model("Org2")

# Metabolites
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular A
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular A
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product
bio2e = cb.Metabolite("bio2e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio2 = cb.Reaction("exchangeBio2", lower_bound=-1000, upper_bound=1000)
exBio2.add_metabolites({
    bio2e: -1.0
})

"""#Transporters """
# Transport A_e → A_c
transport = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transport.add_metabolites({
    B_e: -1,
    B_c: 1
})
# Transport A_e → A_c
transport_bio2 = cb.Reaction("T_Bio2", lower_bound=0, upper_bound=1000)
transport_bio2.add_metabolites({
    bio2e: 1,
    bio2: -1
})

"""#Internal RXNs"""
# Biomass reaction: A_c → bio1
biomass2 = cb.Reaction("BiomassOrg2", lower_bound=-1000, upper_bound=1000)
biomass2.add_metabolites({
    B_c: -1,
    bio2: 1
})

# Add reactions to the model
model_IV.add_reactions([exB, transport, transport_bio2,biomass2,  exBio2])

# Set objective
model_IV.objective = "T_Bio2"

# Solve
sol = model_IV.optimize()
print(f"Objective direction: {model_IV.objective.direction}")
print(f"Max biomass: {sol.objective_value}\n")
print(sol.fluxes)

model_IV.summary()

cb.io.save_json_model(model_IV, "sim_IV_2.json")


Objective direction: max
Max biomass: 10.0

Ex_B           -10.0
T_B             10.0
T_Bio2          10.0
BiomassOrg2     10.0
exchangeBio2    10.0
Name: fluxes, dtype: float64


In [19]:
# sit V org 1
# Create model
model_V_1 = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product
bio1e = cb.Metabolite("bio1e", compartment="e")  # extracellular biomass product


"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=0, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=0, upper_bound=1000)
exBio1.add_metabolites({
    bio1e: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})
# Transport B_c → B_e
transport_bio1 = cb.Reaction("T_Bio1", lower_bound=0, upper_bound=1000)
transport_bio1.add_metabolites({
    bio1: -1,
    bio1e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1 + B_c (always creates some B)
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes 2 A_c
    bio1: 1,  # Produces bio1
    B_c: 1   # Always produces 1 B_c as byproduct
})

# Add reactions to the model
model_V_1.add_reactions([exA, exB, transportA, transportB, transport_bio1, biomassA,  exBio1])

# Set objective
model_V_1.objective = "T_Bio1"

# Solve
sol = model_V_1.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_V_1.summary()

cb.io.save_json_model(model_V_1, "sim_V_1.json")


Max biomass: 10.0
Ex_A           -10.0
Ex_B            10.0
T_A             10.0
T_B             10.0
T_Bio1          10.0
BiomassFromA    10.0
exchangeBio1    10.0
Name: fluxes, dtype: float64


In [22]:
# sit V org 2
# Create model
model_V_2 = cb.Model("Org2")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product
bio2e = cb.Metabolite("bio2e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({
    A_e: -1
})
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({
    B_e: -1
})
# Biomass exchange: export bio2
exBio2 = cb.Reaction("exchangeBio2", lower_bound=0, upper_bound=1000)
exBio2.add_metabolites({
    bio2e: -1.0
})

"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1
})
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1
})
# Transport B_c → B_e
transport_bio2 = cb.Reaction("T_Bio2", lower_bound=0, upper_bound=1000)
transport_bio2.add_metabolites({
    bio2: -1,
    bio2e: 1
})

"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio2
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes A_c
    bio2: 1,  # Produces bio2
})
# Biomass reaction from A: B_c → bio2
biomassB = cb.Reaction("BiomassFromB", lower_bound=0, upper_bound=1000)
biomassB.add_metabolites({
    B_c: -1,  # Consumes 2 B_c
    bio2: 1,  # Produces bio2
})

# Add reactions to the model
model_V_2.add_reactions([exA, exB, transportA, transportB, transport_bio2,biomassA, biomassB, exBio2])

# Set objective
model_V_2.objective = "T_Bio2"

# Solve
sol = model_V_2.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_V_2.summary()

cb.io.save_json_model(model_V_2, "sim_V_2.json")


Max biomass: 20.0
Ex_A           -10.0
Ex_B           -10.0
T_A             10.0
T_B            -10.0
T_Bio2          20.0
BiomassFromA    10.0
BiomassFromB    10.0
exchangeBio2    20.0
Name: fluxes, dtype: float64


In [24]:
# sit VI org 1
# Create model
model_VI_1 = model_V_2.copy()

for rxn in model_VI_1.reactions:
    print(rxn)
    
model_VI_1.metabolites.get_by_id("bio2").id = "bio1"
model_VI_1.metabolites.get_by_id("bio2e").id = "bio1e"
model_VI_1.reactions.get_by_id("exchangeBio2").id = "exchangeBio1"
model_VI_1.reactions.get_by_id("T_Bio2").id = "T_Bio1"
model_VI_1.repair()

for rxn in model_VI_1.reactions:
    print(rxn)

model_VI_1.id = "Org1"

cb.io.save_json_model(model_VI_1, "sim_VI_1.json")

Read LP format model from file /tmp/tmpycecy6j9.lp
Reading time = 0.00 seconds
: 6 rows, 16 columns, 26 nonzeros
Ex_A: A_e <=> 
Ex_B: B_e <=> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
T_Bio2: bio2 --> bio2e
BiomassFromA: A_c --> bio2
BiomassFromB: B_c --> bio2
exchangeBio2: bio2e --> 
Ex_A: A_e <=> 
Ex_B: B_e <=> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
T_Bio1: bio1 --> bio1e
BiomassFromA: A_c --> bio1
BiomassFromB: B_c --> bio1
exchangeBio1: bio1e --> 


In [26]:
# sit VI org 2
# Create model
model_VI_2 = model_V_1.copy()
for rxn in model_VI_2.reactions:
	print(rxn)

model_VI_2.metabolites.get_by_id("bio1").id = "bio2"
model_VI_2.metabolites.get_by_id("bio1e").id = "bio2e"
model_VI_2.metabolites.get_by_id("A_e").id = "C_e"
model_VI_2.metabolites.get_by_id("A_c").id = "C_c"
model_VI_2.reactions.get_by_id("exchangeBio1").id = "exchangeBio2"
model_VI_2.reactions.get_by_id("T_A").id = "T_C"
model_VI_2.reactions.get_by_id("T_Bio1").id = "T_Bio2"
model_VI_2.reactions.get_by_id("Ex_A").id = "Ex_C"
model_VI_2.reactions.get_by_id("BiomassFromA").id = "BiomassFromC"
model_VI_2.repair()

for rxn in model_VI_2.reactions:
	print(rxn)

model_VI_2.id = "Org2"

cb.io.save_json_model(model_VI_2, "sim_VI_2.json")

Read LP format model from file /tmp/tmp7b9a7qrd.lp
Reading time = 0.00 seconds
: 6 rows, 14 columns, 24 nonzeros
Ex_A: A_e <=> 
Ex_B: B_e --> 
T_A: A_e <=> A_c
T_B: B_c <=> B_e
T_Bio1: bio1 --> bio1e
BiomassFromA: A_c --> B_c + bio1
exchangeBio1: bio1e --> 
Ex_C: C_e <=> 
Ex_B: B_e --> 
T_C: C_e <=> C_c
T_B: B_c <=> B_e
T_Bio2: bio2 --> bio2e
BiomassFromC: C_c --> B_c + bio2
exchangeBio2: bio2e --> 


In [ ]:
Org 1: Eat A make B
Org 2: Eat B make C
Org 1: Eat C make D
Org 2: Eat D make E

In [28]:
# sit VII org 1
# Create model
model_VII_1 = cb.Model("Org1")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
C_e = cb.Metabolite("C_e", compartment="e")  # extracellular C
C_c = cb.Metabolite("C_c", compartment="c")  # intracellular C
D_e = cb.Metabolite("D_e", compartment="e")  # extracellular D
D_c = cb.Metabolite("D_c", compartment="c")  # intracellular D
bio1 = cb.Metabolite("bio1", compartment="c")  # intracellular biomass product
bio1e = cb.Metabolite("bio1e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({ A_e: -1 })
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({ B_e: -1 })
# Exchange reaction: import A (can import A when available)
exC = cb.Reaction("Ex_C", lower_bound=-10, upper_bound=1000)
exC.add_metabolites({ C_e: -1 })
# Exchange reaction: import/export B
exD = cb.Reaction("Ex_D", lower_bound=-10, upper_bound=1000)
exD.add_metabolites({ D_e: -1 })
# Biomass exchange: export bio1
exBio1 = cb.Reaction("exchangeBio1", lower_bound=0, upper_bound=1000)
exBio1.add_metabolites({
    bio1e: -1.0
})


"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1 })
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1 })
# Transport C_e → C_c
transportC = cb.Reaction("T_C", lower_bound=-1000, upper_bound=1000)
transportC.add_metabolites({
    C_e: -1,
    C_c: 1 })
# Transport D_c → D_e
transportD = cb.Reaction("T_D", lower_bound=-1000, upper_bound=1000)
transportD.add_metabolites({
    D_c: -1,
    D_e: 1 })
# Transport D_c → D_e
transport_bio1 = cb.Reaction("T_Bio1", lower_bound=-1000, upper_bound=1000)
transport_bio1.add_metabolites({
    bio1: -1,
    bio1e: 1 })


"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1
biomassA = cb.Reaction("BiomassFromA", lower_bound=0, upper_bound=1000)
biomassA.add_metabolites({
    A_c: -1,  # Consumes A_c
    bio1: 1,  # Produces bio1
    B_c: 1})
# Biomass reaction from A: B_c → bio1
biomassC = cb.Reaction("BiomassFromC", lower_bound=0, upper_bound=1000)
biomassC.add_metabolites({
    C_c: -1,  # Consumes 2 C_c
    bio1: 1,  # Produces bio1
    D_c: 1})


# Add reactions to the model
model_VII_1.add_reactions([exA, exB, exC, exD, transportA, transportB, transportC, transportD, transport_bio1, biomassA, biomassC, exBio1])

# Set objective
model_VII_1.objective = "T_Bio1"

# Solve
sol = model_VII_1.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_VII_1.summary()

cb.io.save_json_model(model_VII_1, "sim_VII_1.json")


Max biomass: 20.0
Ex_A           -10.0
Ex_B            10.0
Ex_C           -10.0
Ex_D            10.0
T_A             10.0
T_B             10.0
T_C             10.0
T_D             10.0
T_Bio1          20.0
BiomassFromA    10.0
BiomassFromC    10.0
exchangeBio1    20.0
Name: fluxes, dtype: float64


In [30]:
# sit VII org 2
# Create model
model_VII_2 = cb.Model("Org2")

# Metabolites
A_e = cb.Metabolite("A_e", compartment="e")  # extracellular A
A_c = cb.Metabolite("A_c", compartment="c")  # intracellular A
B_e = cb.Metabolite("B_e", compartment="e")  # extracellular B
B_c = cb.Metabolite("B_c", compartment="c")  # intracellular B
C_e = cb.Metabolite("C_e", compartment="e")  # extracellular C
C_c = cb.Metabolite("C_c", compartment="c")  # intracellular C
D_e = cb.Metabolite("D_e", compartment="e")  # extracellular D
D_c = cb.Metabolite("D_c", compartment="c")  # intracellular D
bio2 = cb.Metabolite("bio2", compartment="c")  # intracellular biomass product
bio2e = cb.Metabolite("bio2e", compartment="e")  # extracellular biomass product

"""#Exchanges"""
# Exchange reaction: import A (can import A when available)
exA = cb.Reaction("Ex_A", lower_bound=-10, upper_bound=1000)
exA.add_metabolites({ A_e: -1 })
# Exchange reaction: import/export B
exB = cb.Reaction("Ex_B", lower_bound=-10, upper_bound=1000)
exB.add_metabolites({ B_e: -1 })
# Exchange reaction: import A (can import A when available)
exC = cb.Reaction("Ex_C", lower_bound=-10, upper_bound=1000)
exC.add_metabolites({ C_e: -1 })
# Exchange reaction: import/export B
exD = cb.Reaction("Ex_D", lower_bound=-10, upper_bound=1000)
exD.add_metabolites({ D_e: -1 })
# Biomass exchange: export bio2
exBio2 = cb.Reaction("exchangeBio2", lower_bound=-1000, upper_bound=1000)
exBio2.add_metabolites({
    bio2e: -1.0
})


"""#Transporters"""
# Transport A_e → A_c
transportA = cb.Reaction("T_A", lower_bound=-1000, upper_bound=1000)
transportA.add_metabolites({
    A_e: -1,
    A_c: 1 })
# Transport B_c → B_e
transportB = cb.Reaction("T_B", lower_bound=-1000, upper_bound=1000)
transportB.add_metabolites({
    B_c: -1,
    B_e: 1 })
# Transport C_e → C_c
transportC = cb.Reaction("T_C", lower_bound=-1000, upper_bound=1000)
transportC.add_metabolites({
    C_e: -1,
    C_c: 1 })
# Transport D_c → D_e
transportD = cb.Reaction("T_D", lower_bound=-1000, upper_bound=1000)
transportD.add_metabolites({
    D_c: -1,
    D_e: 1 })
transport_bio2 = cb.Reaction("T_Bio2", lower_bound=-1000, upper_bound=1000)
transport_bio2.add_metabolites({
    bio2: -1,
    bio2e: 1
})



"""# Internal Rxns"""
# Biomass reaction from A: A_c → bio1
biomassB = cb.Reaction("BiomassFromB", lower_bound=0, upper_bound=1000)
biomassB.add_metabolites({
    B_c: -1,  # Consumes B_c
    bio2: 1,  # Produces bio2
    C_c: 1})
# Biomass reaction from A: B_c → bio1
biomassD = cb.Reaction("BiomassFromD", lower_bound=0, upper_bound=1000)
biomassD.add_metabolites({
    D_c: -1,  # Consumes 2 D_c
    bio2: 1 })# Produces bio2 


# Add reactions to the model
model_VII_2.add_reactions([exA, exB, exC, exD, transportA, transportB, transportC, transportD, transport_bio2, biomassB, biomassD, exBio2])

# Set objective
model_VII_2.objective = "T_Bio2"

# Solve
sol = model_VII_2.optimize()
print(f"Max biomass: {sol.objective_value}")
print(sol.fluxes)

model_VII_2.summary()

cb.io.save_json_model(model_VII_2, "sim_VII_2.json")


Max biomass: 20.0
Ex_A             0.0
Ex_B           -10.0
Ex_C            10.0
Ex_D           -10.0
T_A              0.0
T_B            -10.0
T_C            -10.0
T_D            -10.0
T_Bio2          20.0
BiomassFromB    10.0
BiomassFromD    10.0
exchangeBio2    20.0
Name: fluxes, dtype: float64
